In [1]:
import numpy as np
import pandas as pd
import time
import h5pyd
import geopandas as gpd
from dw_tap.data_fetching import getData
from dw_tap.power_output import estimate_power_output
from dw_tap.lom import run_lom

z_turbine = 40 # turbine height in [m]
lat, lon = 39.32856, -89.40238
obstacle_file = "../sites/180-Manual1.geojson"

t_start = time.time()

#Read in data from CSV file instead of hsds + getData
atmospheric_df = pd.read_csv("../data/180_1year_12hourgranularity.csv")
# Temporary (later need to resave csv without index column) 
if "Unnamed: 0" in atmospheric_df.columns:
    atmospheric_df.drop(columns=["Unnamed: 0"], inplace=True)

obstacles_df = gpd.read_file(obstacle_file)
# Leave in only relevant columns
obstacles_df = obstacles_df[["height", "geometry"]]

x1_turbine, y1_turbine = lat, lon
xy_turbine = [np.array([x1_turbine, y1_turbine])]

t_lom_start = time.time()
predictions_df = \
    run_lom(atmospheric_df, obstacles_df, xy_turbine, z_turbine)
t_lom = time.time() - t_lom_start
print('Running LOM: %.2f (s)' % t_lom)

t_power_start = time.time()
kw, above_curve, below_curve, above_curve_list, below_curve_list = \
    estimate_power_output(predictions_df, atmospheric_df["temp"], atmospheric_df["pres"])  
t_power = time.time() - t_power_start
print('Power estimation: %.2f (s)' % t_power)

t_total = time.time()-t_start
print('Total time: %.2f (s)' % t_total)

2022-10-24 15:42:56.675141: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


LOM time : 0.06  min
Running LOM: 3.55 (s)
Power estimation: 0.02 (s)
Total time: 3.60 (s)


In [15]:
##### Cells below can be used for demonstration purposes

In [9]:
# atmospheric_df

In [10]:
# obstacles_df #height is in meters; geometry has polygons in (lat,lon) pairs

In [11]:
# xy_turbine # location of turbine in lat,lon

In [12]:
# z_turbine # hub height in meters 

In [14]:
# predictions_df = \
#     run_lom(atmospheric_df, obstacles_df, xy_turbine, z_turbine)

In [13]:
# predictions_df